In [5]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('/content/drive/MyDrive/Luxury_Beauty/luxury_beauty_cLeaned.csv')

df.head()

,review,vote
0,this handcream has a beautiful fragrance it do...,1
1,wonderful hand lotion for seriously dry skin s...,1
2,best hand cream around silky thick soaks in a...,1
3,thanks five stars,1
4,great hand lotion soaks right in and leaves s...,1


# Using BERT


In [ ]:
!pip install tensorflow_text

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
# from official.nlp import optimization

In [55]:
from sklearn.model_selection import train_test_split

X = df['vote'].astype(int)
y = df['review'].apply(str)
x_train, x_val, y_train, y_val = train_test_split(X, y)

In [121]:
def create_dataset(inputs, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((labels, inputs))
    dataset = dataset.cache().shuffle(len(inputs)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

##building the model

In [128]:
from tensorflow import keras
from keras.layers import Input, Dense, Dropout

class FineTuneBERT(keras.Model):
  def __init__(self, units, ratio):
    super(FineTuneBERT, self).__init__()

    self.preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
    self.encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/2')

    self.dense = Dense(units, activation='relu')
    self.dropout = Dropout(ratio)
    self.out = Dense(1, activation='sigmoid')

  def call(self, x):
    # x, label = inputs
    x = self.preprocessor(x)
    x = self.encoder(x)
    x = x['pooled_output']
    x = self.dense(x)
    x = self.dropout(x)

    return self.out(x)

In [129]:
######### HyperParameters ############
UNITS = 512
LEARNING_RATE = 1e-3
DROPOUT = .3
EPOCHS = 10
BATCH_SIZE = 32
######################################

train_dataset = create_dataset(x_train, y_train, BATCH_SIZE)
val_dataset = create_dataset(x_val, y_val, BATCH_SIZE)

model = FineTuneBERT(UNITS, DROPOUT)

In [130]:
model.compile(
    optimizer=keras.optimizers.AdamW(LEARNING_RATE),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=keras.metrics.BinaryAccuracy()
)

In [131]:
tensorboard = keras.callbacks.TensorBoard('/content/drive/MyDrive/Luxury_Beauty/BERT/logs')
checkpoints = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Luxury_Beauty/BERT/checkpoints')
earlystopping = keras.callbacks.EarlyStopping(patience=1)

model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=[tensorboard, checkpoints, earlystopping]
)

Epoch 1/10
804/804 [==============================] - ETA: 0s - loss: 0.4071 - binary_accuracy: 0.8188

804/804 [==============================] - 1931s 2s/step - loss: 0.4071 - binary_accuracy: 0.8188 - val_loss: 0.3669 - val_binary_accuracy: 0.8371
Epoch 2/10
804/804 [==============================] - ETA: 0s - loss: 0.3822 - binary_accuracy: 0.8304

804/804 [==============================] - 1945s 2s/step - loss: 0.3822 - binary_accuracy: 0.8304 - val_loss: 0.3807 - val_binary_accuracy: 0.8326


we here used a small BERT and as we just see we got only 83% which is a slight improvment from the base 80% model (a model that will predict the major class only!) 
so to solve this proplem we can use another BERT Model